# 決定木 Decision Tree による要因探索とメトリック記録

製造工程データに決定木を適用し、説明変数・因子がどのように製造品の良品・不良品に影響しているかを分析します。最初は普通のScikit Learnを使用し決定木で学習します。Azure Machine Learning service のメトリック記録機能を使って、作業内容を記録することも示します。

## Azure Machine Learning service
Azure Machine Learning service の資産管理の仕組みを利用して、メトリック、データセット、モデルを保存します。
### Python SDK インポート

In [ ]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

In [ ]:
print("Azure ML SDK Version: ", azureml.core.VERSION)

### ワークスペースへの接続

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

### 実験の開始

In [ ]:
experiment=Experiment(ws, "telhandson2")
run = experiment.start_logging()

# 学習データの準備

In [ ]:
import pandas as pd
import os

df = pd.read_csv("Factory.csv")

In [ ]:
df.head(10)

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(["Quality","ID"],axis=1)
y = df["Quality"].values

# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=100,stratify=y)

# 決定木 Decision Tree による学習
[Scikit-Learn Decision Tree](https://scikit-learn.org/stable/modules/tree.html) による機械学習モデリング

In [ ]:
from sklearn import tree
criterion='gini'
max_depth=4
clf = tree.DecisionTreeClassifier(max_depth=max_depth, criterion=criterion)
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import (roc_curve, auc, accuracy_score)

pred = clf.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
print("AUC", auc(fpr, tpr))
print("正解率", accuracy_score(pred, y_test))

In [ ]:
%matplotlib inline

import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr,label='ROC curve (area = %.2f)'%auc(fpr, tpr))
plt.title("ROC curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid(True)
plt.savefig('ROC.png')
plt.show()

## メトリックの記録

In [ ]:
# パラメータ
run.log("木の分割基準", criterion)
run.log("木の深さ", max_depth)

# 精度
run.log("AUC", auc(fpr, tpr))
run.log("正解率", accuracy_score(pred, y_test))
run.log_image("ROC", "ROC.png")

### モデル登録

In [ ]:
import pickle
filename = 'DT-model.pkl'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
run.upload_file("outputs/"+filename, filename)

# モデル登録

In [ ]:
from azureml.core.model import Model

model = run.register_model(model_name='DT-sklearn', 
                           model_path = 'outputs/' + filename,
                           tags = {'area': "sklearnによる決定木", 'type': "分類 Classification", 'run_id' : run.id}
                          )

In [ ]:
print(model.name, model.version, model.tags)

## 実験完了

In [ ]:
run.complete()

In [ ]:
run.get_details()

In [ ]:
run